Author: Geoff Boeing <br />
Web: http://geoffboeing.com/2015/03/chaos-theory-logistic-map/  <br />
Date: 2015-03-30 <br />
Description: This notebook demonstrates how to create Return Maps, or Poincaré Plots, to visualize system attractors and differentiate random noise from chaos. Return Maps embed 1-dimensional data (like time series) from a discrete dynamical system into 2- or 3-dimensional state space by plotting the value at time t vs the value at time t+1 (vs the value at time t+2, if it's a 3-D plot). I demonstrate how to make animated GIFs to pan and zoom around these 3-D plots.

In [1]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image as PIL_Image
from images2gif import writeGif
import glob
import IPython.display as IPdisplay

In [2]:
# for documentation of this function, see chaos-logistic-model.ipynb
def logistic_model(generations=20, growth_rate_min=0.5, growth_rate_max=4.0, growth_rate_steps=7, pop_initial=0.5):
    growth_rate_min = float(growth_rate_min)
    growth_rate_max = float(growth_rate_max) - 0.0000000001
    growth_rates = np.arange(growth_rate_min, growth_rate_max, (growth_rate_max - growth_rate_min) / growth_rate_steps)
    pops = pd.DataFrame(columns=growth_rates, index=range(generations))
    pops.iloc[0] = pop_initial
    for rate in pops.columns:
        pop = pops[rate]
        for t in range(generations - 1):
            pop[t + 1] = pop[t] * rate * (1 - pop[t])
    return pops

In [3]:
# for documentation of this function, see chaos-logistic-poincare-plots.ipynb
def get_poincare_points_3d(pops, discard_gens):
    if discard_gens > 0:
        discard_gens = np.arange(0, discard_gens)
        pops = pops.drop(labels=pops.index[discard_gens])
    li = []
    for rate in pops.columns:
        for label, row in pops.iterrows():
            if label < len(pops)-2:
                li.append((row[rate], pops[rate][label + 1], pops[rate][label + 2]))
    return pd.DataFrame(li, columns=['x', 'y', 'z'])

In [4]:
############################################################################
# show a 3-D poincare plot as an animated gif
############################################################################
gif_filename = 'demo-pan-rotate-zoom'

# run the logistic model for 1000 generations for the growth rate parameter 3.99
r = 3.99
xyz = get_poincare_points_3d(logistic_model(1000, r, 4, 1), 1)

# create the figure and axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# scatterplot the data in 3-D
ax.scatter(xyz['x'], xyz['y'], xyz['z'], s=3, marker='o', facecolors='m', edgecolors='m', alpha=0.1, depthshade=True)

# remove all ticks
ax.tick_params(reset=True, axis='both', which='both', pad=0, width=0, length=0,
               bottom='off', top='off', left='off', right='off', 
               labelbottom='off', labeltop='off', labelleft='off', labelright='off')

# create 36 frames for the animated gif
steps = 36

# a viewing perspective is composed of an elevation, distance, and azimuth
# define the range of values for the distance of the viewing perspective
min_dist = 7.
max_dist = 10.
dist_range = np.arange(min_dist, max_dist, (max_dist-min_dist)/steps)

# define the range of values for the elevation of the viewing perspective
min_elev = 10.
max_elev = 60.
elev_range = np.arange(max_elev, min_elev, (min_elev-max_elev)/steps)

# pan down, rotate around, and zoom out
for azimuth in range(0, 360, 360/steps):
    ax.azim = float(azimuth)
    ax.elev = elev_range[int(azimuth/(360./steps))]
    ax.dist = dist_range[int(azimuth/(360./steps))]
    fig.suptitle('elev=' + str(round(ax.elev,1)) + ', azim=' + str(round(ax.azim,1)) + ', dist=' + str(round(ax.dist,1)))
    plt.savefig('images/' + gif_filename + '/img' + str(azimuth).zfill(3) + '.png')
    
# don't display the static plot
plt.close()

# create an animated gif of all the 3-D plot perspectives then display it inline
images = [PIL_Image.open(image) for image in glob.glob('images/' + gif_filename + '/*.png')]
file_path_name = 'images/' + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.2)
IPdisplay.Image(url=file_path_name)

In [5]:
############################################################################
# show the 3-D plot as an animated gif
############################################################################
gif_filename = 'logistic-3d-return-map'

# run the logistic model for 1000 generations for the growth rate parameter 3.99
r = 3.99
xyz = get_poincare_points_3d(logistic_model(1000, r, 4, 1), 1)

# create the figure and axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# scatterplot the data in 3-D
ax.scatter(xyz['x'], xyz['y'], xyz['z'], s=3, marker='o', facecolors='m', edgecolors='m', alpha=0.1, depthshade=True)

# set axis labels, limits, and remove all ticks
ax.set_xlabel('t',  labelpad=0)
ax.set_ylabel('t + 1',  labelpad=0)
ax.set_zlabel('t + 2',  labelpad=0)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_zlim(0, 1)
ax.tick_params(reset=True, axis='both', which='both', pad=0, width=0, length=0,
               bottom='off', top='off', left='off', right='off', 
               labelbottom='off', labeltop='off', labelleft='off', labelright='off')

# configure the initial viewing perspective to look straight down at the x-y plane
ax.elev = 89.9
ax.azim = 270.1
ax.dist = 10.0

# sweep the perspective down and rotate to reveal the 3-D structure of the strange attractor
for n in range(0, 75):
    if n > 9 and n < 13:
        ax.elev = ax.elev-0.5 #start by panning down slowly
    if n > 12 and n < 27:
        ax.elev = ax.elev-1.0 #pan down faster
    if n > 26 and n < 51:
        ax.elev = ax.elev-1.5
        ax.azim = ax.azim+1 #pan down faster and start to rotate
    if n > 50 and n < 55:
        ax.elev = ax.elev-1.0
        ax.azim = ax.azim+1 #pan down slower and rotate same speed
    if n > 54 and n < 64:
        ax.elev = ax.elev-0.5
        ax.azim = ax.azim+1 #pan down slowly and rotate same speed
    if n > 63 and n < 67:
        ax.elev = ax.elev-0.2
        ax.azim = ax.azim+0.5 #end by panning/rotating slowly to stopping position
        
    # lastly, add a figure title to each plot then save the figure to the disk
    fig.suptitle(u'Poincaré Plot of 3D state space, r=' + str(r), fontsize=12, x=0.5, y=0.85)
    plt.savefig('images/' + gif_filename + '/img' + str(n).zfill(3) + '.png', bbox_inches='tight')

# don't display the static plot
plt.close()

# create an animated gif of all the 3-D plot perspectives then display it inline
images = [PIL_Image.open(image) for image in glob.glob('images/' + gif_filename + '/*.png')]
file_path_name = 'images/' + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.1)
IPdisplay.Image(url=file_path_name)

In [6]:
############################################################################
# show a 3-D plot of chaos vs random noise as an animated gif
############################################################################
gif_filename = 'chaos-vs-random-return-map'

# run the logistic model for 1000 generations for the growth rate parameter 3.99
r = 3.99
chaos_points = get_poincare_points_3d(logistic_model(1000, r, 4, 1), 1)

# create a random time series data set
random_pops = pd.DataFrame([random.random() for _ in range(0, 1000)], columns=['value'])
rand_points = get_poincare_points_3d(random_pops, 1)

# create the figure and axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# scatterplot the data in 3-D
chaos_scatter = ax.scatter(chaos_points['x'], chaos_points['y'], chaos_points['z'], 
                           s=3, marker='o', facecolors='m', edgecolors='m', alpha=0.2, depthshade=True)
rand_scatter = ax.scatter(rand_points['x'], rand_points['y'], rand_points['z'], 
                          s=3, marker='o', facecolors='c', edgecolors='c', alpha=0.5, depthshade=True)

# set axis labels and remove all ticks
ax.set_xlabel('t',  labelpad=0)
ax.set_ylabel('t + 1',  labelpad=0)
ax.set_zlabel('t + 2',  labelpad=0)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_zlim(0, 1)
ax.tick_params(reset=True, axis='both', which='both', pad=0, width=0, length=0,
               bottom='off', top='off', left='off', right='off', 
               labelbottom='off', labeltop='off', labelleft='off', labelright='off')

# add a legend
lgd = ax.legend((chaos_scatter, rand_scatter), ('chaos', 'random'), loc=3, frameon=True, framealpha=1)

# configure the initial viewing perspective to look straight down at the x-y plane
ax.elev = 89.9
ax.azim = 270.1
ax.dist = 10.0

# sweep the perspective down and rotate to reveal the 3-D structure of the strange attractor
for n in range(0, 75):
    if n > 9 and n < 13:
        ax.elev = ax.elev-0.5 #start by panning down slowly
    if n > 12 and n < 27:
        ax.elev = ax.elev-1.0 #pan down faster
    if n > 26 and n < 51:
        ax.elev = ax.elev-1.5
        ax.azim = ax.azim+1 #pan down faster and start to rotate
    if n > 50 and n < 55:
        ax.elev = ax.elev-1.0
        ax.azim = ax.azim+1 #pan down slower and rotate same speed
    if n > 54 and n < 64:
        ax.elev = ax.elev-0.5
        ax.azim = ax.azim+1 #pan down slowly and rotate same speed
    if n > 63 and n < 67:
        ax.elev = ax.elev-0.2
        ax.azim = ax.azim+0.5 #end by panning/rotating slowly to stopping position
        
    # add a figure title to each plot then save the figure to the disk
    fig.suptitle(u'Poincaré Plot of 3D state space', fontsize=12, x=0.5, y=0.85)
    plt.savefig('images/' + gif_filename + '/img' + str(n).zfill(3) + '.png', bbox_inches='tight')

# don't display the static plot
plt.close()

# create an animated gif of all the 3-D plot perspectives then display it inline
images = [PIL_Image.open(image) for image in glob.glob('images/' + gif_filename + '/*.png')]
file_path_name = 'images/' + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.1)
IPdisplay.Image(url=file_path_name)